In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')

True

In [ ]:
from langsmith import utils

utils.tracing_is_enabled()

True

# Build a Chatbot

LLM Chatbot --> RAG Chatbot(RAG 추가) --> Agent Chatbot(Tools 추가)

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [11]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-151a8c61-9eb2-485f-9962-10ace3ff3a31-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

In [6]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 11, 'total_tokens': 39, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-d965acf3-9c24-4bcb-a867-266fe8e541df-0', usage_metadata={'input_tokens': 11, 'output_tokens': 28, 'total_tokens': 39})

상기 예제의 문제점은 이전 대화를 기억하지 못한다는 점

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. How can I help you, Bob?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 35, 'total_tokens': 48}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9f90291b-4df9-41dc-9ecf-1ee1081f4490-0', usage_metadata={'input_tokens': 35, 'output_tokens': 13, 'total_tokens': 48})

## Message History

In [12]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory # 대화 내용을 기억하면서 LLM를 호출
from langsmith import traceable

store = {}

# 이 함수의 실행 과정(입력, 출력, 실행 시간 등)을 기록합니다.
# 검색 관련 작업으로 분류하여 대시보드나 로그에서 관련 데이터를 체계적으로 보여줍니다.
@traceable(run_type="retriever")
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [13]:
config = {"configurable": {"session_id": "abc2"}}

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hi Bob! How can I assist you today?'

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob! How can I help you today?'

Great! Our chatbot now remembers things about us. If we change the config to reference a different `session_id`, we can see that it starts the conversation fresh.

In [9]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?"

## Prompt templates
`MessagesPlaceholder`: invoke() 수행에서 해당 내용을 키(변수 이름)의 value로 변경

In [6]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [7]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hi Bob! How can I assist you today?'

RunnableWithMessageHistory - 대화형 애플리케이션에서 메시지의 히스토리를 관리하면서 LLM을 실행

In [14]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [15]:
config = {"configurable": {"session_id": "abc5"}}

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello, Jim! How can I assist you today?'

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim.'

Awesome! Let's now make our prompt a little bit more complicated. Let's assume that the prompt template now looks something like this:

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

Note that we have added a new `language` input to the prompt. We can now invoke the chain and pass in a language of our choice.

In [19]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages", # 입력이 여러개 있는데 도대체 묻고 싶은 입력은 뭐에요?
)

In [21]:
config = {"configurable": {"session_id": "abc11"}}

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Todd.'

## Managing Conversation History

LLM context window 크기 한계와 비용 문제로 적절한 History 관리가 필요 - trim_messages

In [24]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True, # system message는 보존
    allow_partial=False, # 부분적 메세지 보존 허용은 불허
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

To  use it in our chain, we just need to run the trimmer before we pass the `messages` input to our prompt. 

아래 예제는 Trim으로 인하여 이름을 기억하지 못함

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

# assign은 데이터 조작 작업을 수행한 데이터를 Runnable로 만듬
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but I don't have access to your personal information. How can I assist you today?"

아래 예제처럼 메세지 기억 범위내에 있는 경우, 정확한 답변 가능

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2?"'

Let's now wrap this in the Message History

In [27]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to that information. How can I assist you today?"

messages를 plus 하지 않아서 모든 기록이 사라져 아래와 같은 결과가 유도됨

In [29]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked a math problem yet. Feel free to ask any math-related question you have, and I'll be happy to help you with it."

## Streaming

In [30]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Todd|!| Sure|,| here|'s| a| joke| for| you|:| Why| couldn|'t| the| bicycle| find| its| way| home|?| Because| it| lost| its| bearings|!| 😄||